In [218]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import os
#print(os.listdir("../input"))

import time

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [219]:
#make wider graphs
sns.set(rc={'figure.figsize':(12,5)});
plt.figure(figsize=(12,5));

<Figure size 1200x500 with 0 Axes>

# Для этой части я использую первые 10 миллионов строк из обучающего набора (так как полные наборы слишком велики для этого ядра) и полные тестовые данные.
# Примечание: в ноутбуке я иногда называю загрузки конверсиями. То есть is_attributed == 1 означает, что клик привел к конверсии.

In [220]:
#import first 10,000,000 rows of train and all test data
train = pd.read_csv('/home/jupyter/datasphere/project/data/TalkingData_AdTracking/train.csv', nrows=10_000_000)
test = pd.read_csv('/home/jupyter/datasphere/project/data/TalkingData_AdTracking/test.csv')

In [221]:
train

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0
2,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0
3,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0
4,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0
...,...,...,...,...,...,...,...,...
9999995,64609,21,1,11,128,2017-11-07 00:12:03,NaN,0
9999996,46277,12,1,15,178,2017-11-07 00:12:03,NaN,0
9999997,149939,20,1,19,259,2017-11-07 00:12:03,NaN,0
9999998,23256,3,1,27,280,2017-11-07 00:12:03,NaN,0


In [222]:
test

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00
...,...,...,...,...,...,...,...
18790464,18790464,99442,9,1,13,127,2017-11-10 15:00:00
18790465,18790465,88046,23,1,37,153,2017-11-10 15:00:00
18790466,18790467,81398,18,1,17,265,2017-11-10 15:00:00
18790467,18790466,123236,27,1,13,122,2017-11-10 15:00:00


ip, app, device, os и channel на самом деле являются категориальными переменными, закодированными как целые числа. Установим их как категории для анализа.

In [223]:
variables = ['ip', 'app', 'device', 'os', 'channel']
for v in variables:
    train[v] = train[v].astype('category')
    test[v]=test[v].astype('category')

Преобразовать временные метки в тип дата/время.

In [224]:
#set click_time and attributed_time as timeseries
train['click_time'] = pd.to_datetime(train['click_time'])
train['attributed_time'] = pd.to_datetime(train['attributed_time'])
test['click_time'] = pd.to_datetime(test['click_time'])

#set as_attributed in train as a categorical
train['is_attributed']=train['is_attributed'].astype('category')

Теперь проведем быстрый осмотр основных статистических показателей обучающих и тестовых данных

In [225]:
train.describe()

/tmp/ipykernel_7845/3346399198.py:1: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.

/tmp/ipykernel_7845/3346399198.py:1: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



,ip,app,device,os,channel,click_time,attributed_time,is_attributed
count,10000000.0,10000000.0,10000000.0,10000000.0,10000000.0,10000000,18717,10000000.0
unique,68740.0,332.0,940.0,292.0,170.0,29943,15698,2.0
top,73516.0,12.0,1.0,19.0,245.0,2017-11-06 16:05:10,2017-11-06 23:36:23,0.0
freq,51711.0,1291185.0,9381146.0,2410148.0,793105.0,1261,6,9981283.0
first,NaN,NaN,NaN,NaN,NaN,2017-11-06 14:32:21,2017-11-06 16:00:47,NaN
last,NaN,NaN,NaN,NaN,NaN,2017-11-07 00:12:03,2017-11-07 15:59:53,NaN


In [226]:
import pandas as pd
import plotly.express as px

cols = ['ip', 'app', 'device', 'os', 'channel']
uniques = [len(train[col].unique()) for col in cols]
unique_df = pd.DataFrame({'Признак': cols, 'Количество уникальных значений': uniques})

# Сортируем по убыванию
unique_df = unique_df.sort_values('Количество уникальных значений', ascending=False)

# График с учётом порядка категорий
fig = px.bar(
    unique_df,
    x='Признак',
    y='Количество уникальных значений',
    text='Количество уникальных значений',
    color='Признак',
    category_orders={'Признак': unique_df['Признак'].tolist()}
)

# Логарифмическая шкала по оси Y
fig.update_yaxes(type='log', title='log(количество уникальных значений)')

# Настройки компоновки
fig.update_layout(
    title='Уникальные значения для каждого признака (из 10,000,000 образцов)',
    xaxis_title='Признак',
    width=1000,
    height=600,
    font=dict(size=13),
    showlegend=False
)

fig.update_traces(textposition='outside')
fig.show()

Быстрая проверка, чтобы убедиться, что значения NaN в 'attributed_time' встречаются только для образцов без конверсии. Проверка, что количество значений 'attributed_time' совпадает с количеством конвертированных кликов.

In [227]:
# Проверяем, что 'attributed_time' не является Null для всех значений, которые привели к скачиванию (т.е. is_attributed == 1)
train[['attributed_time', 'is_attributed']][train['is_attributed']==1].describe()

/tmp/ipykernel_7845/1868893430.py:2: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



,attributed_time,is_attributed
count,18717,18717.0
unique,15698,1.0
top,2017-11-06 23:36:23,1.0
freq,6,18717.0
first,2017-11-06 16:00:47,NaN
last,2017-11-07 15:59:53,NaN


In [228]:
# установить click_id как категориальный тип для более чистого статистического представления
test['click_id']=test['click_id'].astype('category')
test.describe()

/tmp/ipykernel_7845/245231234.py:3: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



,click_id,ip,app,device,os,channel,click_time
count,18790469.0,18790469.0,18790469.0,18790469.0,18790469.0,18790469.0,18790469
unique,18790469.0,93936.0,417.0,1985.0,395.0,178.0,21603
top,0.0,5348.0,9.0,1.0,19.0,107.0,2017-11-10 09:00:10
freq,1.0,182522.0,2872176.0,17360269.0,4334532.0,1214650.0,1466
first,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-10 04:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-10 15:00:00


**Быстрые заметки и наблюдения**:
- Всего 18717 значений attributed_time. Это означает, что только 18,717 из 10,000,000 кликов привели к скачиванию. Это менее 0.2%!
- Есть IP-адреса, которые генерируют более 50 тысяч кликов. Странно, что один IP-адрес может кликать так часто за период всего в 4 дня. Возможно, IP-адрес кодирует не идентификатор устройства, а идентификатор сети? (исследуем это ниже)
- Первый клик в обучающей выборке был 2017-11-06 14:32:21. Тестовые клики начинаются 2017-11-10. Согласно спецификации данных, обучающая выборка охватывает 4-дневный период. Это означает, что обучающие и тестовые данные не перекрываются, а тестовые данные взяты на следующий день после окончания обучающих данных.
- Обучающие данные отсортированы по временной метке. (поэтому пакеты, извлеченные по порядку, охватывают ограниченный временной промежуток)
- 2017-11-06 был понедельником. 2017-11-10 был пятницей. Т.е. Обучающая выборка - пн-чт, Тестовая - пятница
- В тестовой выборке нет пропущенных данных. Пропущенные значения в обучающей выборке встречаются только для attributed_time, где нет значения из-за отсутствия скачивания приложения.

Только небольшая часть кликов привела к скачиванию приложения:

In [229]:
# Вычисляем долю скачиваний
mean = (train.is_attributed.values == 1).mean()

# Готовим данные для графика
plot_data = pd.DataFrame({
    'Категория': ['Фрод (1)', 'Не фрод (0)'],
    'Доля': [mean, 1 - mean]
})

# Строим интерактивную столбчатую диаграмму
fig = px.bar(
    plot_data,
    x='Категория',
    y='Доля',
    text='Доля',
    color='Категория',
)

# Форматируем ось Y и подписи в процентах
fig.update_yaxes(title='Доля', tickformat='.2%')
fig.update_traces(
    texttemplate='%{y:.2%}',  # выводим y-значение в формате процента
    textposition='outside'
)

# Оформление и размеры
fig.update_layout(
    title='Фрод vs Пользователь',
    width=600,
    height=600,
    showlegend=False,
    font=dict(size=13)
)

fig.show()

## Исследование количества IP-адресов. Проверка наличия скачиваний для множественных IP.

На данном этапе я пытался разобраться, что на самом деле кодируют IP-адреса. Моё первоначальное понимание, что IP-адреса специфичны для пользователей, не подтвердилось при более детальном анализе. Если IP-адрес повторяется слишком много раз, является ли это ботом? Это также не подтвердилось, так как повторяющиеся IP-адреса всё равно приводят к конверсиям. Смотрите ниже:

In [230]:
#временная таблица для просмотра IP-адресов с их частотой появления
temp = train['ip'].value_counts().reset_index(name='counts')
temp.columns = ['ip', 'counts']
temp[:10]

,ip,counts
0,73516,51711
1,73487,51215
2,5314,35073
3,5348,35004
4,53454,25381
5,105560,23289
6,100275,23070
7,114276,22774
8,201182,22719
9,105475,22047


In [231]:
#добавляем временный признак количества IP ('counts') в таблицу train, чтобы проверить, есть ли конверсии у IP с большим количеством кликов
train= train.merge(temp, on='ip', how='left')

In [232]:
#проверяем топ-10 значений
train[train['is_attributed']==1].sort_values('counts', ascending=False)[:10]

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,counts
7912764,73516,34,1,19,243,2017-11-06 23:28:13,2017-11-07 01:13:45,1,51711
289189,73516,19,3033,21,213,2017-11-06 16:05:33,2017-11-06 16:05:52,1,51711
3350587,73516,72,1,18,101,2017-11-06 17:46:37,2017-11-07 09:46:44,1,51711
3416495,73516,10,1,25,377,2017-11-06 17:50:33,2017-11-06 18:50:59,1,51711
6331848,73516,29,1,14,213,2017-11-06 22:36:25,2017-11-07 02:10:05,1,51711
7543731,73516,155,1,19,213,2017-11-06 23:18:10,2017-11-07 15:07:40,1,51711
5144374,73516,9,1,37,234,2017-11-06 20:49:25,2017-11-06 20:53:19,1,51711
3651360,73516,107,1,19,171,2017-11-06 18:05:12,2017-11-06 23:44:44,1,51711
6368416,73516,10,1,13,377,2017-11-06 22:38:04,2017-11-06 22:52:43,1,51711
2073360,73516,9,1,13,134,2017-11-06 16:52:12,2017-11-06 16:52:20,1,51711


In [233]:
train[train['is_attributed']==1].ip.describe()

count     18717
unique    16112
top       73487
freq         56
Name: ip, dtype: int64

IP-адреса с высокой частотой появления также имеют конверсии. До 56 загрузок для одного IP-адреса. Каждый IP-адрес, вероятно, представляет собой сеть с множеством устройств.

In [234]:
# Преобразуем 'is_attributed' обратно в числовой формат для расчета пропорций
train['is_attributed']=train['is_attributed'].astype(int)

# Коэффициенты конверсии для 300 самых популярных IP-адресов

In [235]:
proportion = train[['ip', 'is_attributed']].groupby('ip', as_index=False).mean().sort_values('is_attributed', ascending=False)
counts = train[['ip', 'is_attributed']].groupby('ip', as_index=False).count().sort_values('is_attributed', ascending=False)
merge = counts.merge(proportion, on='ip', how='left')
merge.columns = ['ip', 'click_count', 'prop_downloaded']

# Создаем график с двумя осями Y используя plotly
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Добавляем данные для количества кликов
fig.add_trace(
    go.Scatter(x=merge[:300].index, y=merge[:300]['click_count'], 
               name="Количество кликов",
               line=dict(width=3)),  # Делаем линию жирнее
    secondary_y=False,
)

# Добавляем данные для пропорции скачиваний
fig.add_trace(
    go.Scatter(x=merge[:300].index, y=merge[:300]['prop_downloaded'], 
               name="Доля скачиваний"),
    secondary_y=True,
)

# Обновляем layout
fig.update_layout(
    title_text="Коэффициенты конверсии для 300 самых популярных IP-адресов",
    xaxis_title="IP-адреса",
    showlegend=True
)

# Обновляем метки осей
fig.update_yaxes(title_text="Количество кликов", secondary_y=False)
fig.update_yaxes(title_text="Доля скачиваний", secondary_y=True)

fig.show()

Конверсии имеют высокую вариативность и не демонстрируют явной корреляции с популярностью IP-адреса.

# Конверсии по приложениям
 
 Проверим 100 самых популярных приложений по количеству кликов:

In [236]:
proportion = train[['app', 'is_attributed']].groupby('app', as_index=False).mean().sort_values('is_attributed', ascending=False)
counts = train[['app', 'is_attributed']].groupby('app', as_index=False).count().sort_values('is_attributed', ascending=False)
merge = counts.merge(proportion, on='app', how='left')
merge.columns = ['app', 'click_count', 'prop_downloaded']

# Создаем график с двумя осями Y используя plotly
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Добавляем данные для количества кликов
fig.add_trace(
    go.Scatter(x=merge[:100].index, y=merge[:100]['click_count'], 
               name="Количество кликов",
               line=dict(width=3)),  # Делаем линию жирнее
    secondary_y=False,
)

# Добавляем данные для пропорции скачиваний
fig.add_trace(
    go.Scatter(x=merge[:100].index, y=merge[:100]['prop_downloaded'], 
               name="Доля скачиваний"),
    secondary_y=True,
)

# Обновляем layout
fig.update_layout(
    title_text="Коэффициенты конверсии для 100 самых популярных приложений",
    xaxis_title="Приложения",
    showlegend=True
)

# Обновляем метки осей
fig.update_yaxes(title_text="Количество кликов", secondary_y=False)
fig.update_yaxes(title_text="Доля скачиваний", secondary_y=True)

fig.show()

Наблюдается значительная разница в количестве кликов по приложениям - от минимального значения в 1 клик до максимального около 13 миллионов. Пропорция конверсий колеблется сильнее при уменьшении количества кликов, так как каждый дополнительный клик оказывает большее влияние на значение пропорции. В целом, для приложений с количеством кликов в тысячах коэффициент конверсии остается в пределах 0.0001 - 0.0015. Для менее популярных приложений наблюдаются более широкие колебания этого показателя.

# Конверсии по операционным системам
 Анализ 100 самых популярных операционных систем по количеству кликов

In [237]:
proportion = train[['os', 'is_attributed']].groupby('os', as_index=False).mean().sort_values('is_attributed', ascending=False)
counts = train[['os', 'is_attributed']].groupby('os', as_index=False).count().sort_values('is_attributed', ascending=False)
merge = counts.merge(proportion, on='os', how='left')
merge.columns = ['os', 'click_count', 'prop_downloaded']

# Создаем график с двумя осями Y используя plotly
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Добавляем данные для количества кликов
fig.add_trace(
    go.Scatter(x=merge[:100].index, y=merge[:100]['click_count'], 
               name="Количество кликов",
               line=dict(width=3)),  # Делаем линию жирнее
    secondary_y=False,
)

# Добавляем данные для пропорции скачиваний
fig.add_trace(
    go.Scatter(x=merge[:100].index, y=merge[:100]['prop_downloaded'], 
               name="Доля скачиваний"),
    secondary_y=True,
)

# Обновляем layout
fig.update_layout(
    title_text="Коэффициенты конверсии для 100 самых популярных операционных систем",
    xaxis_title="Операционные системы",
    showlegend=True
)

# Обновляем метки осей
fig.update_yaxes(title_text="Количество кликов", secondary_y=False)
fig.update_yaxes(title_text="Доля скачиваний", secondary_y=True)

fig.show()

Наблюдается аналогичная картина. Для операционных систем с количеством кликов в тысячах коэффициент конверсии остается в узких пределах от 0.0006 до 0.003. Однако при уменьшении количества кликов на операционную систему, коэффициент конверсии начинает колебаться более значительно.

# Конверсии по устройствам
 
 Устройства распределены крайне неравномерно - самое популярное устройство используется почти в 94% случаев. Для этого устройства доля скачиваний составила 0.001326 (0.13%).

In [238]:
proportion = train[['device', 'is_attributed']].groupby('device', as_index=False).mean().sort_values('is_attributed', ascending=False)
counts = train[['device', 'is_attributed']].groupby('device', as_index=False).count().sort_values('is_attributed', ascending=False)
merge = counts.merge(proportion, on='device', how='left')
merge.columns = ['device', 'click_count', 'prop_downloaded']

print('Count of clicks and proportion of downloads by device:')
print(merge)

Count of clicks and proportion of downloads by device:
    device  click_count  prop_downloaded
0        1      9381146         0.001326
1        2       456617         0.000208
2     3032       104393         0.000000
3        0        46476         0.091961
4       59         1618         0.001236
..     ...          ...              ...
935   1044            1         1.000000
936   1036            1         0.000000
937   1016            1         0.000000
938   1014            1         0.000000
939   3545            1         0.000000

[940 rows x 3 columns]


### Conversions by Channel


In [239]:
proportion = train[['channel', 'is_attributed']].groupby('channel', as_index=False).mean().sort_values('is_attributed', ascending=False)
counts = train[['channel', 'is_attributed']].groupby('channel', as_index=False).count().sort_values('is_attributed', ascending=False)
merge = counts.merge(proportion, on='channel', how='left')
merge.columns = ['channel', 'click_count', 'prop_downloaded']

# Создаем фигуру с двумя осями Y
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Добавляем графики
fig.add_trace(
    go.Scatter(x=merge[:100].index, y=merge[:100]['click_count'], 
               name="Количество кликов",
               line=dict(width=3)),  # Делаем линию жирнее
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=merge[:100].index, y=merge[:100]['prop_downloaded'], 
               name="Доля скачиваний"),
    secondary_y=True,
)

# Обновляем layout
fig.update_layout(
    title_text="Коэффициенты конверсии для 100 самых популярных каналов",
    xaxis_title="Каналы",
    showlegend=True
)

# Обновляем метки осей
fig.update_yaxes(title_text="Количество кликов", secondary_y=False)
fig.update_yaxes(title_text="Доля скачиваний", secondary_y=True)

fig.show()


Похоже, что есть несколько пиков для каналов с разумным количеством кликов, но в целом паттерн сохраняется таким же, как и для категорий выше.

# Анализ временных паттернов
 
Округлим время клика до часа, чтобы выявить возможные почасовые паттерны.

 Для этой части анализа нельзя использовать первые n строк из обучающих данных, так как они организованы по времени. 
 Чтобы получить общее представление о паттерне, будем использовать случайную выборку из 100000 строк обучающего набора, 
 предоставленную организаторами.

In [255]:
train_smp = pd.read_csv('/home/jupyter/datasphere/project/data/TalkingData_AdTracking/train_sample.csv')
train_smp

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0
...,...,...,...,...,...,...,...,...
99995,124883,11,1,19,122,2017-11-09 13:25:41,NaN,0
99996,85150,9,1,13,244,2017-11-07 11:25:43,NaN,0
99997,18839,3,1,13,19,2017-11-08 11:38:42,NaN,0
99998,114276,15,1,12,245,2017-11-08 17:55:21,NaN,0


In [256]:
# Преобразуем столбцы click_time и attributed_time в формат временных рядов
train_smp['click_time'] = pd.to_datetime(train_smp['click_time'])
train_smp['attributed_time'] = pd.to_datetime(train_smp['attributed_time'])

In [257]:
# округляем время до ближайшего часа
train_smp['click_rnd']=train_smp['click_time'].dt.round('H')

# Проверяем почасовые паттерны
# Создаем фигуру с двумя подграфиками
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Добавляем график частоты кликов
fig.add_trace(
    go.Scatter(
        x=train_smp[['click_rnd','is_attributed']].groupby(['click_rnd']).count().index,
        y=train_smp[['click_rnd','is_attributed']].groupby(['click_rnd']).count()['is_attributed'],
        name="Частота кликов",
        line=dict(width=3),  # Делаем линию жирнее
    ),
    secondary_y=False
)

# Добавляем график коэффициента конверсии
fig.add_trace(
    go.Scatter(
        x=train_smp[['click_rnd','is_attributed']].groupby(['click_rnd']).mean().index,
        y=train_smp[['click_rnd','is_attributed']].groupby(['click_rnd']).mean()['is_attributed'],
        name="Коэффициент конверсии",
        line=dict(width=3),  # Делаем линию жирнее
    ),
    secondary_y=True
)

# Обновляем layout
fig.update_layout(
    title_text="Почасовая частота кликов и коэффициент конверсии",
    xaxis_title="Время",
    showlegend=True
)

# Обновляем метки осей
fig.update_yaxes(title_text="Количество кликов", secondary_y=False)
fig.update_yaxes(title_text="Коэффициент конверсии", secondary_y=True)

fig.show()

В коэффициентах конверсии не наблюдается четкого почасового паттерна, однако есть определенная закономерность в частоте кликов в зависимости от времени суток.
 
Давайте извлечем час дня из каждого дня как отдельный признак и посмотрим общий тренд (объединим 4 дня по часам).

In [258]:
# Извлекаем час как признак из времени клика
train_smp['click_hour']=train_smp['click_time'].dt.hour

In [259]:
train_smp.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,click_rnd,click_hour
0,87540,12,1,13,497,2017-11-07 09:30:38,NaT,0,2017-11-07 10:00:00,9
1,105560,25,1,17,259,2017-11-07 13:40:27,NaT,0,2017-11-07 14:00:00,13
2,101424,12,1,19,212,2017-11-07 18:05:24,NaT,0,2017-11-07 18:00:00,18
3,94584,13,1,13,477,2017-11-07 04:58:08,NaT,0,2017-11-07 05:00:00,4
4,68413,12,1,1,178,2017-11-09 09:00:09,NaT,0,2017-11-09 09:00:00,9


Давайте проверим количество кликов по часам:

In [260]:
# Создаем график с двумя осями Y
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Добавляем столбчатую диаграмму частоты кликов
fig.add_trace(
    go.Bar(
        x=train_smp[['click_hour','is_attributed']].groupby(['click_hour']).count().index,
        y=train_smp[['click_hour','is_attributed']].groupby(['click_hour']).count()['is_attributed'],
        name="Частота кликов"
    ),
    secondary_y=False
)

# Добавляем линейный график частоты кликов
fig.add_trace(
    go.Scatter(
        x=train_smp[['click_hour','is_attributed']].groupby(['click_hour']).count().index,
        y=train_smp[['click_hour','is_attributed']].groupby(['click_hour']).count()['is_attributed'],
        name="Частота кликов (линейный)",
        line=dict(width=5),  # Делаем линию жирнее
    ),
    secondary_y=False
)

# Обновляем layout
fig.update_layout(
    title_text="Почасовая частота кликов",
    xaxis_title="Час",
    showlegend=True
)

# Обновляем метки осей
fig.update_yaxes(title_text="Количество кликов", secondary_y=False)

fig.show()

И количество конверсий по часам:

In [261]:
# Создаем график с двумя осями Y
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Добавляем столбчатую диаграмму конверсий
fig.add_trace(
    go.Bar(
        x=train_smp[['click_hour','is_attributed']].groupby(['click_hour']).mean().index,
        y=train_smp[['click_hour','is_attributed']].groupby(['click_hour']).mean()['is_attributed'],
        name="Конверсия (столбчатая)",
    ),
    secondary_y=False
)

# Добавляем линейный график конверсий
fig.add_trace(
    go.Scatter(
        x=train_smp[['click_hour','is_attributed']].groupby(['click_hour']).mean().index,
        y=train_smp[['click_hour','is_attributed']].groupby(['click_hour']).mean()['is_attributed'],
        name="Конверсия (линейный)",
        line=dict(width=5)
    ),
    secondary_y=False
)

# Обновляем layout
fig.update_layout(
    title_text="Почасовая конверсия",
    xaxis_title="Час",
    showlegend=True
)

# Обновляем метки осей
fig.update_yaxes(title_text="Коэффициент конверсии", secondary_y=False)

fig.show()

Давайте наложим два графика друг на друга, чтобы увидеть, есть ли между ними какая-либо корреляция

In [262]:
# Создаем график с двумя осями Y
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Добавляем линейный график конверсий
fig.add_trace(
    go.Scatter(
        x=train_smp[['click_hour','is_attributed']].groupby(['click_hour']).mean().index,
        y=train_smp[['click_hour','is_attributed']].groupby(['click_hour']).mean()['is_attributed'],
        name="Доля конверсий",
        line=dict(width=3)
    ),
    secondary_y=False
)

# Добавляем линейный график количества кликов
fig.add_trace(
    go.Scatter(
        x=train_smp[['click_hour','is_attributed']].groupby(['click_hour']).count().index,
        y=train_smp[['click_hour','is_attributed']].groupby(['click_hour']).count()['is_attributed'],
        name="Количество кликов",
        line=dict(width=3)
    ),
    secondary_y=True
)

# Обновляем layout
fig.update_layout(
    title_text="Зависимость конверсии и количества кликов от времени",
    xaxis_title="Час",
    showlegend=True
)

# Обновляем метки осей
fig.update_yaxes(title_text="Доля конверсий", secondary_y=False)
fig.update_yaxes(title_text="Количество кликов", secondary_y=True)

fig.show()

Пропорции могут быть более надежными при оценке на полных данных. С случайной выборкой трудно определить из-за высокой вариативности, особенно для часов с низким количеством кликов. То есть, чем меньше кликов/конверсий, тем шире погрешность оценки коэффициента конверсии. (см. ниже)

In [308]:
# 1. Группируем и считаем средний коэффициент конверсии и стандартную ошибку
summary = (
    train_smp
    .groupby('click_hour')['is_attributed']
    .agg(['mean', 'count', 'std'])
    .reset_index()
    .rename(columns={'mean': 'conversion_rate'})
)
summary['se'] = summary['std'] / np.sqrt(summary['count'])

# 2. Строим интерактивный барплот с ошибками
fig = px.bar(
    summary,
    x='click_hour',
    y='conversion_rate',
    error_y='se',
    color='click_hour',
    color_continuous_scale='temps',
    labels={
        'click_hour': 'Час',
        'conversion_rate': 'Коэффициент конверсии'
    },
    title='Коэффициент конверсии по часам'
)

# Добавляем подписи часов на оси X
fig.update_xaxes(
    ticktext=[f'{h:02d}' for h in range(24)],
    tickvals=list(range(24))
)

# Убираем шкалу цвета справа
fig.update_layout(coloraxis_showscale=False)

fig.show()

# Анализ времени атрибуции
Полезно изучить детали конверсий, которые действительно произошли.
 Давайте посмотрим, сколько времени прошло от клика по рекламе до скачивания приложения.

In [250]:
train_smp['timePass']= train_smp['attributed_time']-train_smp['click_time']
train_smp[train_smp['is_attributed']==1][:15]

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,click_rnd,click_hour,timePass
284,224120,19,0,29,213,2017-11-08 02:22:13,2017-11-08 02:22:38,1,2017-11-08 02:00:00,2,0 days 00:00:25
481,272894,10,1,7,113,2017-11-08 06:10:05,2017-11-08 06:10:37,1,2017-11-08 06:00:00,6,0 days 00:00:32
1208,79001,19,0,0,213,2017-11-07 09:54:22,2017-11-07 11:59:05,1,2017-11-07 10:00:00,9,0 days 02:04:43
1341,131029,19,0,0,343,2017-11-09 10:58:46,2017-11-09 11:52:01,1,2017-11-09 11:00:00,10,0 days 00:53:15
1412,40352,19,0,0,213,2017-11-07 22:19:03,2017-11-08 01:55:02,1,2017-11-07 22:00:00,22,0 days 03:35:59
1666,48733,35,1,18,274,2017-11-07 12:25:50,2017-11-07 13:10:30,1,2017-11-07 12:00:00,12,0 days 00:44:40
1771,330861,35,1,22,21,2017-11-08 18:54:44,2017-11-08 22:39:52,1,2017-11-08 19:00:00,18,0 days 03:45:08
1917,309576,5,1,32,113,2017-11-09 08:47:51,2017-11-09 08:47:55,1,2017-11-09 09:00:00,8,0 days 00:00:04
3914,220571,71,1,25,3,2017-11-08 04:35:21,2017-11-08 04:37:46,1,2017-11-08 05:00:00,4,0 days 00:02:25
3992,240051,35,1,19,21,2017-11-08 08:07:13,2017-11-08 09:46:42,1,2017-11-08 08:00:00,8,0 days 01:39:29


In [251]:
train_smp['timePass'].describe()

count                          227
mean     0 days 01:14:59.572687224
std      0 days 02:25:05.392661298
min                0 days 00:00:02
25%         0 days 00:00:52.500000
50%                0 days 00:03:18
75%         0 days 01:21:27.500000
max                0 days 12:52:21
Name: timePass, dtype: object

 Время от клика до покупки варьируется от 4 секунд до почти 20 часов.

 4 секунды кажутся слишком коротким временем для принятия решения. Такой пользователь, вероятно, либо уже видел рекламу ранее, либо был знаком с продуктом через другие каналы.

 Это может означать, что реклама была кликнута несколько раз, но только один клик был засчитан как конверсия? Или пользователь кликнул на рекламу с конкретным намерением скачать? (Например, если канал - это что-то вроде поиска Google, реклама могла быть кликнута при просмотре результатов поиска, и приложение было скачано немедленно, потому что это именно то, что пользователь собирался сделать сразу)

 Возникают вопросы для исследования:
    - Насколько точно отслеживаются конверсии? Как связаны клики и загрузки? Что происходит при загрузке после нескольких кликов? Есть ли способ идентифицировать вероятно одних и тех же пользователей (по IP, устройству и т.д.)?

# Проверка полного набора обучающих данных (первые 10 миллионов записей)
 Дополнительная проверка временного интервала между кликом и конверсией на полном наборе данных:

In [252]:
# Проверка первых 10 миллионов записей из полного набора обучающих данных
train['timePass']= train['attributed_time']-train['click_time']
train['timePass'].describe()

count                        18717
mean     0 days 03:48:02.893732970
std      0 days 05:29:41.396810741
min                0 days 00:00:00
25%                0 days 00:01:26
50%                0 days 00:25:03
75%                0 days 06:34:14
max                0 days 23:52:38
Name: timePass, dtype: object

Минимальное время от клика до загрузки практически мгновенное. Как это возможно? Очевидно, что это не результат решения человека, принятого после первого просмотра рекламы.